# 🚀 Credit Approval ML Pipeline

> **Clean Architecture + MLOps-Ready Hybrid Architecture**

Bu notebook, **modüler `.py` dosyalarını** kullanarak pipeline'ı çalıştırır. 
Google Colab ile tam uyumludur.

---

## 1. Environment Setup

In [ ]:
# ============================================
# 1.1 Mount Google Drive (Colab only)
# ============================================

import sys
import os
import warnings
warnings.filterwarnings('ignore')

print("🔄 Checking environment...")

IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    print("🌐 Google Colab detected")
    try:
        from google.colab import drive
        drive.mount('/content/drive', force_remount=True)
        
        # Change to project directory
        # IMPORTANT: Verify this path matches your Drive folder
        PROJECT_PATH = '/content/drive/MyDrive/credit-approval'
        
        if os.path.exists(PROJECT_PATH):
            os.chdir(PROJECT_PATH)
            print(f"✅ Changed directory to: {PROJECT_PATH}")
            if PROJECT_PATH not in sys.path:
                sys.path.append(PROJECT_PATH)  # Add to python path
        else:
            print(f"⚠️ Warning: Path not found: {PROJECT_PATH}")
            print("   Please upload the project to Drive.")
            print(f"   Current directory: {os.getcwd()}")
            
    except Exception as e:
        print(f"❌ Error mounting Drive: {e}")
else:
    print("💻 Local environment detected")
    PROJECT_PATH = os.getcwd()
    print(f"   Current directory: {PROJECT_PATH}")

In [ ]:
# ============================================
# 1.2 Install Dependencies
# ============================================

if IN_COLAB:
    print("📦 Installing dependencies...")
    !pip install -r requirements.txt
else:
    print("ℹ️ Skipping pip install (Local run)")

## 2. Run Pipeline

In [ ]:
# ============================================
# 2.1 Import Pipeline Modules
# ============================================

try:
    from src.core.config import PipelineConfig
    from src.pipelines.training_pipeline import TrainingPipeline
    from src.core.logger import setup_logger

    print("✅ Pipeline modules imported successfully")
except ImportError as e:
    print(f"❌ Import Error: {e}")
    print("   Check if 'src' folder exists and is in python path.")

In [ ]:
# ============================================
# 2.2 Configure Pipeline
# ============================================

config = PipelineConfig(
    optuna_trials=30,      # Number of optimization trials
    cv_folds=5,            # Cross-validation folds
    use_gpu=True,          # Use GPU if available
    output_dir="ml_pipeline_output" 
)

logger = setup_logger().logger
print(f"📋 Config loaded: {config.project_name} v{config.version}")

In [ ]:
# ============================================
# 2.3 Run Training Pipeline
# ============================================

pipeline = TrainingPipeline(
    config=config,
    logger=logger,
    optimize=True
)

try:
    results = pipeline.run()
    print(f"\n🏆 Training Complete! Best Model: {results['best_model']}")
except Exception as e:
    print(f"❌ Execution failed: {e}")

## 3. Results Analysis

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Show plots
plot_files = [
    'model_selection_final.png',
    'feature_importance.png',
    'business_impact_analysis.png'
]

plt.figure(figsize=(20, 10))
for i, plot_file in enumerate(plot_files):
    path = f"{config.output_dir}/plots/{plot_file}"
    if os.path.exists(path):
        img = mpimg.imread(path)
        plt.subplot(1, 3, i+1)
        plt.imshow(img)
        plt.axis('off')
        plt.title(plot_file)
plt.show()